In [1]:
import tensorflow as tf
import numpy as np


char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)
seq_data = ['base', 'word', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']


def make_batch(seq_data):
    input_batch = []
    target_batch = []

    for seq in seq_data:
        input = [num_dic[n] for n in seq[1:]]
        target = num_dic[seq[0]]
        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target)
    return input_batch, target_batch


learning_rate = 0.01
n_hidden = 128
total_epoch = 30
n_input = n_class = dic_len

X = tf.placeholder(tf.float32, [None, None, n_input])
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

outputs = tf.transpose(outputs, [1, 0, 2])# n_hidden의 계층이 맨 뒤로 오도록
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, Y: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

print('Train finished')

prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy],
                                 feed_dict={X: input_batch, Y: target_batch})

predict_words = []
for idx, val in enumerate(seq_data):
    first_char = char_arr[predict[idx]]
    predict_words.append(first_char + val[1:])

print('\n=== Predition ===')
print('input:', [' '+ w[1:] for w in seq_data])
print('prediction:', predict_words)
print('ACCURACY:', accuracy_val)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Epoch: 0001 cost = 3.539886
Epoch: 0002 cost = 2.774084
Epoch: 0003 cost = 1.720592
Epoch: 0004 cost = 0.854450
Epoch: 0005 cost = 0.651148
Epoch: 0006 cost = 0.455489
Epoch: 0007 cost = 0.585434
Epoch: 0008 cost = 0.229154
Epoch: 0009 cost = 0.169450
Epoch: 0010 cost = 0.085436
Epoch: 0011 cost = 0.047532
Epoch: 0012 cost = 0.023845
Epoch: 0013 cost = 0.010758
Epoch: 0014 cost = 0.240559
Epoch: 0015 cost = 0.004698
Epoch: 0016 cost = 0.004848
Epoch: 0017 cost = 0.110395
Epoch: 0018 cost = 0.012531
Epoch: 0019 cost = 0.006408
Epoch: 0020 cost = 0.003399
Epoch: 0021 cost = 0.000907
Epoch: 0022 cost = 0.000535
Epoch: 0023 cost = 0.000431
Epoch: 0024 cost = 0.000034
Epoch: 0025 cost = 0.002117
Epoch: 0026 cost = 0.000070
Epoch: 0027 cost = 0.004722
Epoch: 0028 